# Grating Couplers

Grating couplers are simply components of a photonic circuit that use refraction to couple light into a waveguide. By utilizing refraction, we can avoid have to use expensive specialized equipment to send light directly through the edge of the chip and we get the added versatility of being able input light to any location on the chip.

![Image of an electromagnetic wave](https://raw.githubusercontent.com/BYUCamachoLab/Photonics-Bootcamp/main/book/images/Notebook_Images/grating_coupler_3d.png)
{cite}`Cheng_Mao_Li_Han_Fu_2020`

Grating couplers have a periodic structure. The rows in the image above are alternating mediums of two different refractive indexes. By alternating the refractive index, the grating coupler creates interference patterns in the light which result in a forward propogating wave. The specifics of how this refraction works can be described by Hyugen's Fresnel principle and Bragg's law.

# How does it work?

In [1]:
import gdsfactory as gf

In [2]:
@gf.cell
def simple_grating_coupler(num_teeth = 24, period = 0.2, tooth_width = 0.1, tooth_height = 2, taper_length = 5):
    c1 = gf.Component("grating tooth")
    tooth = c1 << gf.components.straight(length=tooth_width, width = tooth_height)
    
    c2 = gf.Component("grating teeth")
    a_ref = c2.add_array(
        c1, columns=num_teeth, rows=1, spacing=[period, 1]
    )
    
    c3 = gf.Component("taper")
    taper_ref = c3 << gf.components.straight(length=taper_length, width = tooth_height)

    taper_ref.movex(num_teeth * period)

    c4 = gf.Component("final")
    c4.add_ref(c2)

    c4.add_ref(c3)

    return c4

simple_grating_coupler(num_teeth=15)

2023-05-22 12:42:15.741 | WARNING  | gdsfactory.pdk:get_active_pdk:563 - No active PDK. Activating generic PDK.



Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03 \x00\x00\x02X\x08\x02\x00\x00\x00\x15\x14\x15\'\…

simple_grating_coupler__173251f5: uid 6d0d12ea, ports [], references ['grating teeth_1', 'taper_1'], 0 polygons


# GDS

In [1]:
import gdsfactory as gf
from shapely.geometry.polygon import Polygon

@gf.cell
def focusing_grating_coupler(grating_period = 0.7, width = 0.35, taper_length = 20, num_teeth = 20 , port_width = 1):
    
    def concentric_rings(num_circles = num_teeth):
        c = gf.Component("Concentric circles")
        for i in range(num_circles):
            initial_radius = taper_length + grating_period - 0.5 * width
            c.add_ref(gf.components.ring(radius = initial_radius + i * grating_period, width = width))
        return c
    
    component = gf.Component("focusing_grating_coupler")
    component.add_ref(concentric_rings())

    center_circle = gf.components.circle(radius=taper_length)
    component.add_ref(center_circle)

    triangle_length = taper_length + 2 * num_teeth
    triangle_height = 20
    
    component = gf.geometry.trim(component=component, domain=[[0, port_width/2], [0, -port_width/2], [-triangle_length, -triangle_height], [-triangle_length, triangle_height]])

    component2 = gf.Component("waveguide")
    rect_ref = gf.components.straight(width = 10)
    component2.add_ref(rect_ref)
    component.add_port(name = "o1", center = (0,0), width=port_width, orientation = 0, layer=1)

    return component

c = focusing_grating_coupler(grating_period=0.7, width = 0.35, taper_length = 10, num_teeth = 24, port_width=0.5)
c

2023-05-22 13:49:16.385 | WARNING  | gdsfactory.pdk:get_active_pdk:563 - No active PDK. Activating generic PDK.



Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03 \x00\x00\x02X\x08\x02\x00\x00\x00\x15\x14\x15\'\…

focusing_grating_couple_f701ae8b: uid 1834fdec, ports ['o1'], references [], 25 polygons
